In [ ]:
# 1. SETUP & CONFIGURATION

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from PIL import Image
import kagglehub

path = kagglehub.dataset_download("birdy654/cifake-real-and-ai-generated-synthetic-images")
print("Path to dataset files:", path)

# Configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 64
LEARNING_RATE = 0.001
EPOCHS = 10
DATA_DIR = path

print(f" Device set to: {DEVICE}")

In [ ]:
# 2. DATA PREPROCESSING & LOADING

# using a pre-trained ResNet
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

print(" Loading Datasets...")
train_data = datasets.ImageFolder(os.path.join(DATA_DIR, 'train'), transform=transform_train)
test_data = datasets.ImageFolder(os.path.join(DATA_DIR, 'test'), transform=transform_test)

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

class_names = train_data.classes
print(f" Data Loaded. Classes: {class_names}")
print(f" Training Samples: {len(train_data)}")
print(f" Testing Samples: {len(test_data)}")



In [ ]:

# 3. MODEL ARCHITECTURE

def build_model():
    """
    using ResNet18, a powerful feature extractor.
    and replace the final layer to output just 2 classes: REAL vs FAKE.
    """
    model = models.resnet18(pretrained=True)

    # Freeze early layers
    for param in model.parameters():
        param.requires_grad = False

    # Replace the final fully connected layer
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(num_ftrs, 128),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(128, 2) # 2 Output Classes
    )

    return model.to(DEVICE)

model = build_model()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=LEARNING_RATE)

print(" Model Architecture Initialized (ResNet18 backbone)")



In [ ]:

# 4. TRAINING LOOP

def train_model(model, train_loader, criterion, optimizer, epochs=10):
    train_losses = []
    train_accs = []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        print(f"\nEpoch {epoch+1}/{epochs}")
        print("-" * 20)

        for images, labels in train_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)

            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total
        train_losses.append(epoch_loss)
        train_accs.append(epoch_acc)

        print(f"Loss: {epoch_loss:.4f} | Accuracy: {epoch_acc:.2f}%")

    return model, train_losses, train_accs

print("🚀 Starting Training Phase...")
trained_model, losses, accuracies = train_model(model, train_loader, criterion, optimizer, EPOCHS)



In [ ]:

# 5. EVALUATION & SAVING

print("\n Evaluating on Test Set...")
trained_model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = trained_model(images)
        _, predicted = torch.max(outputs.data, 1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print("\n Classification Report:")
print(classification_report(all_labels, all_preds, target_names=class_names))

torch.save(trained_model.state_dict(), 'deepfake_detector_resnet.pth')
print(" Model saved as 'deepfake_detector_resnet.pth'")

